# Computação Bioinspirada 2024-02

Este repositório contém o trabalho da Tópicos Avançados em Ciências de Computação II
(Computação Bioinspirada, no semestre 2024-02).

O trabalho consiste em trabalhar com um conjunto de dados multirrótulo onde cada instância representa uma sequência de
proteína. Cada rótulo (classe) corresponde a uma localização subcelular e as proteínas podem estar presentes
simultaneamente em dois ou mais compartimentos celulares. O conjunto de dados possui seis localizações
subcelulares: Proteínas do Capsídeo Viral, Proteínas da Membrana Celular do Hospedeiro, Proteínas do Retículo
Endoplasmático do Hospedeiro, Proteínas do Citoplasma do Hospedeiro, Proteínas do Núcleo do Hospedeiro e
Proteínas Secretadas. As colunas representam os códigos de Gene Ontology (relacionados à função da proteína),
com valores que indicam a frequência do código para cada proteína. As seis últimas colunas indicam a presença
(1) ou ausência (0) da proteína em cada uma das localizações subcelulares mencionadas.

Desenvolvemos um modelo de classificação multirrótulo usando **redes neurais artificiais**.

Existem dois conjuntos de dados: um de vírus e um de plantas.


### Imports 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

2024-12-04 09:19:29.241945: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 09:19:30.174568: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-04 09:19:30.183196: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-04 09:19:33.354943: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
random_seed = 42
tf.random.set_seed(random_seed)
tf.keras.utils.set_random_seed(random_seed)

### Lendo os dados dos datasets e separando em treino e teste

In [3]:
plants_dataset = pd.read_csv('./Plants_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]
virus_dataset =  pd.read_csv('./Virus_Dataset_Term_Frequency.tsv', sep='\t', skiprows=1).iloc[:, 1:]

In [ ]:
# Os dois datasets possuem 6 rótulos
NUM_ROTULOS_PLANTAS = 12
NUM_ROTULOS_VIRUS = 6
TAMANHO_TEST = 0.3


# Separar as features e os rótulos para o dataset de plantas
X_plants = plants_dataset.iloc[:, :-NUM_ROTULOS_PLANTAS].values  # Todas as colunas, exceto as últimas 6 e a primeira
y_plants = plants_dataset.iloc[:, -NUM_ROTULOS_PLANTAS:].values  # As últimas 6 colunas (rótulos)


# Separar as features e os rótulos para o dataset de vírus
X_virus = virus_dataset.iloc[:, :-NUM_ROTULOS_VIRUS].values
y_virus = virus_dataset.iloc[:, -NUM_ROTULOS_VIRUS:].values
print(len(X_plants[0]), len(X_virus[0]))

# Dividir dados em treino e teste para o dataset de plantas
X_train_plants, X_test_plants, y_train_plants, y_test_plants = train_test_split(
    X_plants, y_plants, test_size=TAMANHO_TEST, random_state=random_seed
)

# Dividir dados em treino e teste para o dataset de vírus
X_train_virus, X_test_virus, y_train_virus, y_test_virus = train_test_split(
    X_virus, y_virus, test_size=TAMANHO_TEST, random_state=random_seed
)

TypeError: len() takes exactly one argument (2 given)

In [ ]:

# Padronizar os dados
scaler_plants = StandardScaler()
X_train_plants = scaler_plants.fit_transform(X_train_plants)
X_test_plants = scaler_plants.transform(X_test_plants)

scaler_virus = StandardScaler()
X_train_virus = scaler_virus.fit_transform(X_train_virus)
X_test_virus = scaler_virus.transform(X_test_virus)

In [ ]:
# Arquitetura da rede neural
def create_model(input_shape, output_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(output_shape, activation='sigmoid')  # Saída com sigmoid para classificação binária
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Criar e treinar o modelo para plantas
model_plants = create_model(X_train_plants.shape[1], y_train_plants.shape[1])
history_plants = model_plants.fit(X_train_plants, y_train_plants, epochs=50, batch_size=32, validation_split=0.2)

/home/teo/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-11-29 16:32:43.253567: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.1501 - loss: 0.7331 - val_accuracy: 0.2968 - val_loss: 0.5993
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2929 - loss: 0.3144 - val_accuracy: 0.3097 - val_loss: 0.4181
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3943 - loss: 0.1703 - val_accuracy: 0.2774 - val_loss: 0.3404
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4248 - loss: 0.1124 - val_accuracy: 0.2774 - val_loss: 0.3027
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4499 - loss: 0.0835 - val_accuracy: 0.2839 - val_loss: 0.2783
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4670 - loss: 0.0636 - val_accuracy: 0.2903 - val_loss: 0.2605
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4736 - loss: 0.0504 - val_accuracy: 0.2968 - val_loss: 0.2483
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4770 - loss: 0.0406 - val_accuracy: 0.3097 - v

In [ ]:
# Criar e treinar o modelo para vírus
model_virus = create_model(X_train_virus.shape[1], y_train_virus.shape[1])
history_virus = model_virus.fit(X_train_virus, y_train_virus, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step - accuracy: 0.1303 - loss: 0.8235 - val_accuracy: 0.3030 - val_loss: 0.7287
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.3057 - loss: 0.6683 - val_accuracy: 0.3030 - val_loss: 0.6719
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.4147 - loss: 0.5734 - val_accuracy: 0.2727 - val_loss: 0.6252
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5139 - loss: 0.5043 - val_accuracy: 0.3636 - val_loss: 0.5876
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5663 - loss: 0.4487 - val_accuracy: 0.3636 - val_loss: 0.5541
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6310 - loss: 0.4019 - val_accuracy: 0.3636 - val_loss: 0.5245
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6003 - loss: 0.3617 - val_accuracy: 0.3333 - val_loss: 0.5010
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6369 - loss: 0.3274 - val_accuracy: 0.3636 - val_loss: 0.4819

In [ ]:
# Obter previsões para o conjunto de teste de plantas
predictions_plants_test = model_plants.predict(X_test_plants)

# Obter previsões para o conjunto de treino de plantas
predictions_plants_train = model_plants.predict(X_train_plants)


# Obter previsões para o conjunto de teste de vírus
predictions_virus_test = model_virus.predict(X_test_virus)

# Obter previsões para o conjunto de treino de vírus
predictions_virus_train = model_virus.predict(X_train_virus)


# Transformar as probabilidades em rótulos binários (0 ou 1) com um limiar de 0.5
predicted_classes_plants_test = (predictions_plants_test > 0.5).astype(int)
predicted_classes_plants_train = (predictions_plants_train > 0.5).astype(int)
predicted_classes_virus_test = (predictions_virus_test > 0.5).astype(int)
predicted_classes_virus_train = (predictions_virus_train > 0.5).astype(int)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


In [ ]:
def hamming_loss(predictions, actual_values):
    N = len(predictions)
    L = 6

    s = 0

    for i in range(N):
        for j in range(L):
            s += predictions[i][j] ^ actual_values[i][j]
    
    return s / (N * L)

In [ ]:
possibilidades = [
    [predicted_classes_plants_train, y_train_plants, 'dados de treino das plantas'],
    [predicted_classes_plants_test, y_test_plants, 'dados de teste das plantas'],
    [predicted_classes_virus_train, y_train_virus, 'dados de treino dos vírus'],
    [predicted_classes_virus_test, y_test_virus, 'dados de teste dos vírus']
]

In [ ]:
for possibilidade in possibilidades:
    print(f'A hamming loss nos {possibilidade[-1]} foi de {100*hamming_loss(*possibilidade[:-1]):.2f}%')

A hamming loss nos dados de treino das plantas foi de 1.49%
A hamming loss nos dados de teste das plantas foi de 3.02%
A hamming loss nos dados de treino dos vírus foi de 2.95%
A hamming loss nos dados de teste dos vírus foi de 6.10%
